In [4]:
#basic libraries
import numpy as np
import pandas as pd

#statistical tests
from scipy.stats import chi2_contingency, f_oneway
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings


In [5]:
from google.colab import files
uploaded = files.upload()


Saving case_study1.xlsx to case_study1.xlsx


In [8]:
from google.colab import files
uploaded = files.upload()


Saving case_study2.xlsx to case_study2.xlsx


In [10]:
df_case1 = pd.read_excel("case_study1.xlsx")
df_case2 = pd.read_excel("case_study2.xlsx")

In [11]:
df1 = df_case1.copy()
df2 = df_case2.copy()

In [12]:
#remove rows where ciritical feature is missing
df1 = df1.loc[df1['Age_Oldest_TL']!=-99999]

In [13]:
#drop columns with excessive missing values
columns_to_remove = []

for col in df2.columns:
  missing_count = df2[df2[col] == -99999].shape[0]

  if missing_count > 10000:
    columns_to_remove.append(col)

df2 = df2.drop(columns=columns_to_remove)

In [14]:
for col in df2.columns:
  df2 =df2[df2[col]!= -99999]

In [15]:
df = pd.merge(
    df1,
    df2,
    how="inner",
    on="PROSPECTID"
)

df.shape


(42064, 79)

In [16]:
categorical_features = []
numerical_features = []

for col in df.columns:
    if df[col].dtype == 'object':
        categorical_features.append(col)
    else:
        numerical_features.append(col)

categorical_features, numerical_features


(['MARITALSTATUS',
  'EDUCATION',
  'GENDER',
  'last_prod_enq2',
  'first_prod_enq2',
  'Approved_Flag'],
 ['PROSPECTID',
  'Total_TL',
  'Tot_Closed_TL',
  'Tot_Active_TL',
  'Total_TL_opened_L6M',
  'Tot_TL_closed_L6M',
  'pct_tl_open_L6M',
  'pct_tl_closed_L6M',
  'pct_active_tl',
  'pct_closed_tl',
  'Total_TL_opened_L12M',
  'Tot_TL_closed_L12M',
  'pct_tl_open_L12M',
  'pct_tl_closed_L12M',
  'Tot_Missed_Pmnt',
  'Auto_TL',
  'CC_TL',
  'Consumer_TL',
  'Gold_TL',
  'Home_TL',
  'PL_TL',
  'Secured_TL',
  'Unsecured_TL',
  'Other_TL',
  'Age_Oldest_TL',
  'Age_Newest_TL',
  'time_since_recent_payment',
  'num_times_delinquent',
  'max_recent_level_of_deliq',
  'num_deliq_6mts',
  'num_deliq_12mts',
  'num_deliq_6_12mts',
  'num_times_30p_dpd',
  'num_times_60p_dpd',
  'num_std',
  'num_std_6mts',
  'num_std_12mts',
  'num_sub',
  'num_sub_6mts',
  'num_sub_12mts',
  'num_dbt',
  'num_dbt_6mts',
  'num_dbt_12mts',
  'num_lss',
  'num_lss_6mts',
  'num_lss_12mts',
  'recent_level_

Manual inspection confirms no post-decision or recovery-based features.
All variables are available at or before loan decision time.

In [18]:
#categorical feature relevance (Chi-Square Test)
target_col = 'Approved_Flag'

categorical_to_test = [
    'MARITALSTATUS',
    'EDUCATION',
    'GENDER',
    'last_prod_enq2',
    'first_prod_enq2'
]

for col in categorical_to_test:
  chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[col],df[target_col]))
  print(f"{col} -> p-value: {pval}")

MARITALSTATUS -> p-value: 3.578180861038862e-233
EDUCATION -> p-value: 2.6942265249737532e-30
GENDER -> p-value: 1.907936100186563e-05
last_prod_enq2 -> p-value: 0.0
first_prod_enq2 -> p-value: 7.84997610555419e-287


In [19]:
vif_df = df[
    [col for col in numerical_features if col not in ['PROSPECTID', 'Approved_Flag']]
]

features_kept = []
vif_data = vif_df.copy()
index=0

for col in vif_df.columns:
  vif = variance_inflation_factor(vif_data.values,index)
  if vif<=6:
    features_kept.append(col)
    index+=1
  else:
    vif_data = vif_data.drop(columns=[col])

/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [20]:
numerical_final =[]

for col in features_kept:
  groups=[]
  for label in df[target_col].unique():
    groups.append(df[df[target_col] == label][col])

  f_stat, p_val = f_oneway(*groups) #see if any similairty exists between list values

  if p_val <= 0.05:
    numerical_final.append(col)

In [23]:
final_features = (
    numerical_final +
    ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']
)

df_final = df[final_features+['Approved_Flag']]
df_final.shape

(42064, 43)

## Feature classification (for documentation / README)
Bureau Features:

- Age_Oldest_TL
- Age_Newest_TL
- time_since_recent_enq
- last_prod_enq2
- first_prod_enq2

Behavioral Features:
- recent_level_of_deliq
- max_recent_level_of_deliq
- time_since_recent_payment

Demographic & Stability Features:
- NETMONTHLYINCOME
- Time_With_Curr_Empr
- EDUCATION
- MARITALSTATUS
- GENDER





In [25]:
df_final.to_csv("clean_data.csv",index=False)

In [26]:
import os
os.listdir()


['.config',
 'clean_data.csv',
 'case_study2.xlsx',
 'case_study1.xlsx',
 'sample_data']

In [27]:
from google.colab import files
files.download("clean_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>